In [1]:
# !pip install -U spacy

In [2]:
# !pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.0.0/ru_core_news_sm-3.0.0.tar.gz


In [3]:
import pandas as pd
import numpy as np

import re

from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist

import torch

from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import pdist, squareform

from tqdm import tqdm

import random

In [4]:
tqdm.pandas()

#EDA


In [5]:
# Connecting Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
folder_path = "/content/drive/My Drive/hackathon/"

## Исследуем файл dealer

In [7]:
dealer = pd.read_csv(folder_path + "marketing_dealer.csv", sep=';', index_col='id')

In [8]:
display(dealer)

name
id                          
1               Moi_vibor_WB
2                      Akson
3                      Bafus
5                  Castorama
6                   Cubatora
7                      Komus
9                  Megastroy
10               OnlineTrade
11                 Petrovich
12                     sdvor
13                  simaLand
14                    VegosM
15            Vse_instrumeni
16                     Vimos
4                  Baucenter
8               Leroy_Merlin
18  Мasterstroy_spb_OZON\r\n
17           Unicleaner_OZON

In [9]:
dealer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 17
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    18 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


## Исследуем файл product

In [10]:
product = pd.read_csv(folder_path + "marketing_product.csv", sep=';', index_col='id')

In [11]:
display(product.sample(5))

Unnamed: 0 article        ean_13  \
id                                      
63          319   189-1  4.680008e+12   
333         123   032-9  4.680008e+12   
5           446   104-5  4.680008e+12   
343          13  034-09  4.680008e+12   
44           24  283-08  4.680008e+12   

                                                  name     cost  \
id                                                                
63   Средство для чистки акриловых поверхностейBath...   114.00   
333  Антисептик лессирующий BiO LASUR / белый люкс ...  2159.00   
5    Универсальное моющее средствоUniversal Prof  к...   601.76   
343  Антисептик лессирующий BiO LASUR / лиственница...   255.00   
44   Средство для мытья полов и стен Multipower "По...    77.94   

     recommended_price  category_id  \
id                                    
63               266.0         46.0   
333             5145.0         31.0   
5               1405.0         56.0   
343              607.0         31.0   
44               182.0          NaN   

                                             ozon_name  \
id                                                       
63   Средство для чистки акриловых ванн и душевых к...   
333  Антисептик лессирующийзащитно-декоративныйPROS...   
5    Профессиональное универсальное моющее средство...   
343  Антисептик лессирующийзащитно-декоративныйPROS...   
44                                                 NaN   

                                               name_1c  \
id                                                       
63   Средство для чистки акриловых ванн и душевых к...   
333  Антисептик лессирующийзащитно-декоративныйPROS...   
5    Профессиональное универсальное моющее средство...   
343  Антисептик лессирующийзащитно-декоративныйPROS...   
44   Средство для мытья полов и стен Multipower "По...   

                                               wb_name  ozon_article  \
id                                                                     
63   Средство для чистки акриловых ванн и душевых к...   448533611.0   
333  Антисептик лессирующийзащитно-декоративныйPROS...   577422226.0   
5    Профессиональное универсальное моющее средство...   449915356.0   
343  Антисептик лессирующийзащитно-декоративныйPROS...   577614089.0   
44                                                 NaN           NaN   

      wb_article ym_article wb_article_td  
id                                         
63   149811050.0      189-1        189-10  
333          NaN        NaN           NaN  
5    149811051.0      104-5           NaN  
343          NaN        NaN           NaN  
44           NaN        NaN           NaN

In [12]:
# Удаляем лишние столбцы


columns_to_drop = ["Unnamed: 0", "article", "ean_13", "cost", "category_id", "ozon_article", "wb_article",
                   "ym_article", "wb_article_td", "ozon_name", "name_1c", "wb_name"]

# Удаление столбцов из датафрейма
product = product.drop(columns=columns_to_drop)

In [13]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 245 to 52
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               494 non-null    object 
 1   recommended_price  491 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.6+ KB


## Исследуем файл key

In [14]:
key = pd.read_csv(folder_path + "marketing_productdealerkey.csv", sep=';', index_col='id')

In [15]:
display(key.sample(5))

key  dealer_id  product_id
id                                    
370   100121743          3         204
1824  674689461         17         431
209   100002946          3         311
265   200544328          3         102
702    90505332          8         320

In [16]:
key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1700 entries, 1 to 2023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.1+ KB


## Исследуем файл price

In [17]:
price = pd.read_csv(folder_path + "marketing_dealerprice.csv", sep=';', parse_dates=['date'], index_col='id')

In [18]:
display(price.sample(5))

product_key   price                                        product_url  \
id                                                                             
15703   200711454   778.0                    https://www.bafus.ru/200711454/   
14936    26397108  2486.0  https://vimos.ru/product/kraska-grunt-fasadnaa...   
16989   530109630   952.0              https://www.ozon.ru/product/530109630   
1847     44182839   250.0  https://www.wildberries.ru/catalog/44182839/de...   
12733   200692789   797.0                    https://www.bafus.ru/200692789/   

                                            product_name       date  dealer_id  
id                                                                              
15703  Просепт ФБС антисептик многофункциональный (10 л) 2023-07-26          3  
14936  Краска-грунт фасадная для плит OSB Prosept Pro... 2023-07-25         16  
16989                   Строительный антисептик  PROSEPT 2023-07-27         18  
1847   Чистящее средство Universal Hard спрей для очи... 2023-07-11          1  
12733  Просепт Паркетный лак алкидно-уретановый для в... 2023-07-24          3

In [19]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20416 entries, 2 to 20570
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_key   20416 non-null  object        
 1   price         20416 non-null  float64       
 2   product_url   20182 non-null  object        
 3   product_name  20416 non-null  object        
 4   date          20416 non-null  datetime64[ns]
 5   dealer_id     20416 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.1+ MB


#Объединяем данные


In [20]:
df = price.merge(dealer, left_on='dealer_id', right_on='id')
df = df.merge(key, left_on='product_key', right_on='key')
df = df.merge(product, left_on='product_id', right_on='id')

In [21]:
df.head(5)

product_key  price                                        product_url  \
0      546227  233.0  https://akson.ru//p/sredstvo_universalnoe_pros...   
1      546227  233.0  https://akson.ru//p/sredstvo_universalnoe_pros...   
2      546227  233.0  https://akson.ru//p/sredstvo_universalnoe_pros...   
3      546227  233.0  https://akson.ru//p/sredstvo_universalnoe_pros...   
4      546227  233.0  https://akson.ru//p/sredstvo_universalnoe_pros...   

                                        product_name       date  dealer_id_x  \
0  Средство универсальное Prosept Universal Spray... 2023-07-11            2   
1  Средство универсальное Prosept Universal Spray... 2023-07-11            2   
2  Средство универсальное Prosept Universal Spray... 2023-07-12            2   
3  Средство универсальное Prosept Universal Spray... 2023-07-13            2   
4  Средство универсальное Prosept Universal Spray... 2023-07-14            2   

  name_x     key  dealer_id_y  product_id  \
0  Akson  546227            2          12   
1  Akson  546227            2          12   
2  Akson  546227            2          12   
3  Akson  546227            2          12   
4  Akson  546227            2          12   

                                              name_y  recommended_price  
0  Универсальное моющее и чистящее средствоUniver...              222.0  
1  Универсальное моющее и чистящее средствоUniver...              222.0  
2  Универсальное моющее и чистящее средствоUniver...              222.0  
3  Универсальное моющее и чистящее средствоUniver...              222.0  
4  Универсальное моющее и чистящее средствоUniver...              222.0

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 17566
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_key        17567 non-null  object        
 1   price              17567 non-null  float64       
 2   product_url        17355 non-null  object        
 3   product_name       17567 non-null  object        
 4   date               17567 non-null  datetime64[ns]
 5   dealer_id_x        17567 non-null  int64         
 6   name_x             17567 non-null  object        
 7   key                17567 non-null  object        
 8   dealer_id_y        17567 non-null  int64         
 9   product_id         17567 non-null  int64         
 10  name_y             17567 non-null  object        
 11  recommended_price  17567 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.7+ MB


### Удаляем лишние данные.

In [23]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["product_key", "product_url", "date", "name_x", "key", "dealer_id_x"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [24]:
# Переименовываем столбец
df = df.rename(columns={'name_y':'original_name'})

# Выявляем типичные отличия в наименовании товара

In [25]:
# Установка максимальной ширины столбцов для отображения полного текста
pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.max_colwidth', None)  # Отображение полного текста в ячейках
df.sample(20)

price  \
11022   848.0   
15511   669.0   
17157  1970.0   
1948    210.0   
11230   368.0   
14000   335.0   
2897    229.0   
15974   244.0   
6899   1052.0   
5082    749.0   
7632    510.0   
16952  2191.0   
373     292.0   
9119    528.0   
11777   882.0   
15885   370.0   
3377    254.0   
4397    435.0   
1781    323.0   
4125    191.0   

                                                                                                      product_name  \
11022                                                     Пенное мыло для дозаторов PROSEPT Diona Flower 5 л 222-5   
15511                                           Средство д/удаления гипсовой пыли PROSEPT Duty White концентрат 5л   
17157                                                    Антисептик невымываемый PROSEPT ECO ULTRA коричневый 20 л   
1948                  Очиститель кожи PROSEPT Duty Leather для изделий из натуральной и искусственной кожи, 400 мл   
11230                                                   Отбеливатель для древесины PROSEPT 50 концентрат, 1л 001-1   
14000                                             Кондиционер для белья PROSEPT Crystal Rinser Концентрат 2л 216-2   
2897                                             Средство для мытья раковин из искусственного камня PROSEPT 500 мл   
15974                        Просепт Professional Optic Crystal средство для мытья стекол, зеркал и пластика (1 л)   
6899                                 Средство Bath DZ для уборки и дезинфекции санитарных комнат 5 л PROSEPT 108-5   
5082                                                            Средство универсальное PROSEPT Universal Spray 5 л   
7632                                                                              ECO UNIVERSAL универсальный анти   
16952                                                 Средство на основе цитрусовых эфирных масел  Duty Citrus, 5л   
373                              Чистящий спрей PROSEPT Universal Anti-dust для люстр и стеклянных изделий, 500 мл   
9119   Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет медовый, 0,6 л 0024-06м   
11777                                     Универсальный антисептик PROSEPT ECO UNIVERSAL готовый состав 10л 015-10   
15885           Просепт Professional Multipower Prof средство усиленного действия для мытья полов концентрат (1 л)   
3377       Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и духовых шкафов (500 мл)   
4397                                                                                Жидкое мыло Prosept Эконом 5 л   
1781                                                                Шпатлевка для плит OSB Proplast Prosept 1,3 кг   
4125                                           CпрейдляочисткикаминныхстеколотсажиикопотиProseptUniversalHard,0,5л   

       dealer_id_y  product_id  \
11022           15         179   
15511            9         122   
17157           18         277   
1948            10         133   
11230           15         240   
14000           15         137   
2897             4          81   
15974            3         201   
6899            15          48   
5082             4          13   
7632             1         281   
16952           13         109   
373             10          18   
9119            15         365   
11777           15         282   
15885            3          24   
3377             3          74   
4397            11         175   
1781            18         234   
4125             5          19   

                                                                                                 original_name  \
11022                  Пенное мыло для дозаторов. Без красителей и ароматизаторовDiona Flowerконцентрат / 5 л    
15511                               Средство для удаления гипсовой пылиDuty Whiteконцентрат 1:10-1:100 / 5 л     
17157                                                                 Антисептик  ECO ULTRA, коричневый / 20 л   


# Выявляем типичные отличия в наименовании товара

In [26]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["price", "dealer_id_y", "recommended_price"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [27]:
%%time
extra_words = (
    # Список слов для удаления из описания
    r'(?:готовый\sсостав|для|концентрат|просепт|prosept|средство|невымываемый|невымываемая|гелеобразный|канистра|всх|типов|бесконтактного|поверхностей|любых|'
    r'чистящее|чистящий|спрей|универсальный|универсальная|универсальное|пэт|жидкое|моющее|гель|чистки|универсаль|мытья|готовый|раствор|ответственных|конструкций|'
    r'концентрированное|professional|готовое|superrubber)'
)

def clean_description(text):
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление чисел в скобках
    text = re.sub(r'\(\d+\)', ' ', text)

    # Вставка пробелов между кириллицей и латиницей
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)

    # Удаление временных меток (часы:минуты)
    text = re.sub(r'\b\d+:\d+\b', ' ', text)

    # Удаление диапазонов чисел, разделенных дефисом или двоеточием
    text = re.sub(r'\b\d+(?::\d+)?[-:]\d+(?::\d+)?\b', ' ', text)

    # Замена фраз вида "2 группа" на "2-ая группа"
    text = re.sub(r'(\d+)\sгруппа', r'\1-ая группа', text)

    # Преобразование объемов из литров в миллилитры и веса из кг в граммы
    text = re.sub(r'(\d+[,.]\d+)\s*л', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} мл", text)
    text = re.sub(r'(\d+[,.]\d+)\s*кг', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} г", text)

    # Добавление пробелов между цифрами и буквами
    text = re.sub(r'(?<=\d)(?=[а-яА-Яa-zA-Z])', ' ', text)
    text = re.sub(r'(?<=[а-яА-Яa-zA-Z])(?=\d)', ' ', text)

    # Удаление серийных номеров или артикулов
    text = re.sub(r'\b\d+-\d+[а-яА-Яa-zA-Z]*\b', ' ', text)

    # Удаление слов, соединенных через "/" или "."
    text = re.sub(r'\b\w+[/\.]\w+(?:[/\.]\w+)?\b', ' ', text)

    # Удаление избыточных слов из списка extra_words
    text = re.sub(extra_words, ' ', text)

    # Удаление пунктуационных знаков и специальных символов
    text = re.sub(r'[,"“”/\.()–;+]', ' ', text)

    # Замена отдельных фраз на альтернативные формы
    text = re.sub(r'бань|баниисауны', 'бани сауны', text)
    text = re.sub(r'литров|литра|литр', 'л', text)
    text = re.sub(r'\b(мыло-гель|гель-мыло)\b', 'мыло', text)
    text = re.sub(r'osb\s+proplast', 'osbproplast', text)
    text = re.sub(r'\bкор\b|\bорех\b', 'коричневый', text)
    text = re.sub(r'eco\s+ultra', 'ecoultra', text)
    text = re.sub(r'гриля\s+и\s+духовых\s+шкафов|удаления\s+пригоревших\s+жиров', 'жироудалитель', text)
    text = re.sub(r'грунт\s+пропиточный', 'грунтовка', text)
    text = re.sub(r'удалитель\s+высолов', 'очиститель фасадов', text)
    text = re.sub(r'с\s+itrus', 'citrus', text)

    # Удаление слова "и"
    text = re.sub(r'\bи\b', ' ', text)

    # Удаление дефисов, окруженных пробелами
    text = re.sub(r'\s-\s', ' ', text)

    # Удаление всех двойных пробелов и пробелов в начале/конце строки
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    text = re.sub(r'eco\s+ultra', 'ultra', text)

    return text

# Применяем функцию к столбцам product_name и original_name
df['product_name_normalized'] = df['product_name'].apply(clean_description)
df['original_name_normalized'] = df['original_name'].apply(clean_description)

CPU times: user 2.21 s, sys: 5.92 ms, total: 2.22 s
Wall time: 2.22 s


In [28]:
df.sample(20, random_state=42)

product_name  \
14843                         Спрей антимикробное д/душ.кабин/акриловых ванн PROSEPT Bath Uni 500мл концентрат   
15011                                                 Средство для удаления цемента Prosept Cement Cleaner 1 л   
6991                              Жидкое средство для стирки PROSEPT CRYSTAL для стирки спортивной одежды, 3 л   
1769                                                      Шпатлевка для плит OSB PROSEPT Proplast 1,3 кг 081-1   
1085                                                                            АнтисептикProseptECOULTRAкор5л   
16322                                Антисептик PROSEPT АНТИПЛЕСЕНЬ, 5 литров, против грибка и плесени (025-5)   
3115                                             Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ   
3379   Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и духовых шкафов (500 мл)   
8256                                     Просепт Bio Lasur антисептик лессирующий защитно-декоративный (2.7 л)   
2830                                                                         Жироудалитель Prosept 0,5 л спрей   
3800                                 Концентрат для мытья полов и стен Полевые цветы PROSEPT Multipower 284-08   
2140                                       Антисептик для внутренних работ Prosept Interior концентрат 1:10 5л   
14922                                 Шампунь для чистки ковров и мягкой мебели Carpet Shampoo. Концентрат, 5л   
4375                                                                   Мыло-гель жидкое д/рук PROSEPT Diona 5л   
3120                                             Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ   
9617                                                              Грунтовка универсальная Prosept 050-10, 10 л   
10237                                            Краска резиновая Prosept SuperRubber коричневый Ral 8017 3 кг   
3402                                                    Средство для удаления пригоревших жиров PROSEPT 500 мл   
12455                                Огнебиозащита для древесины PROSEPT ОГНЕБИО PROF 2-ая группа, 65 л 006-65   
16354                           Универсальный антисептик против грибка и плесени PROSEPT АНТИПЛЕСЕНЬ 5 л 025-5   

       product_id  \
14843          65   
15011         401   
6991          153   
1769          234   
1085          275   
16322         290   
3115          178   
3379           74   
8256          332   
2830           72   
3800           43   
2140          258   
14922         167   
4375          175   
3120          178   
9617          389   
10237         431   
3402           74   
12455         297   
16354         290   

                                                                                                original_name  \
14843  Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л   
15011                                                   Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л   
6991     Жидкое моющее средство для стирки спортивной одежды, обуви и пуховиковCrystal Sportконцентрат / 3 л    
1769                                                                  Шпатлевка для плит OSBProplast / 1,3 кг   
1085                                                                    Антисептик ECO ULTRA коричневый / 5 л   
16322                                     Антисептик против грибка и плесени АНТИПЛЕСЕНЬ готовый состав / 5 л   
3115                жидкое гель-мыло эконом-класса. C ароматом яблокаDiona Citrus Eготовое средство / 5 л ПЭТ   
3379   Средство для чистки гриля и духовых шкафовCooky Grill Gelконцентрированное средство 1:1 – 1:50 / 0,5 л   
8256                                                    Антисептик лессирующий BiO LASUR / белый люкс / 2,7 л   
2830                Средство для чистки гриля и духовых шкафовCooky Grill концентрированное средство / 0,55 л   
3800              

In [29]:
df.sample(20, random_state=41)

product_name  \
1733                                       Антисептик INTERIOR для внутренних работ (концентрат) 1 л PROSEPT 003-1   
1133                                                        Антисептик невымываемый Prosept ECOUltra коричневый 5л   
5736                                               Средство для прочистки труб и стоков Bath Prof, концентрат, 5 л   
8804                       Просепт Professional Prof-DZ универсальное средство с дезинфицирующим эффектом (500 мл)   
14586  Чистящее средство PROSEPT Bath Extra гель концентрат, для удаления ржавчины и минеральных отложений, 750 мл   
12367                                         Просепт Огнебио Prof 1 огнебиозащита для древесины (65 л) бесцветная   
11487                                               Антисептик для влажной древесины PROSEPT BiO, 5 литров (009-5)   
2564                                                           Антисептик трудновымываемый Prosept Есо Exterior 5л   
10719                                                                              Лак для бани и сауны Prosept 2л   
9555                                                                               Просепт Укрепляющий грунт (3 л)   
8053                  Жидкий моющий концентрат для стирки белья PROSEPT Crystal с ароматом белых цветов, 3 л 248-3   
10537                                                                                     Лак для камня Prosept 2л   
2806                    Средство для чистки духовых шкафов и грилей PROSEPT Cooky Grill, концентрат, 550 мл, спрей   
11169                                                       Просепт 50 отбеливатель для древесины (1 л) желтоватый   
6378                                                                              Строительный антисептик  PROSEPT   
5094                                                     Средство Prosept для мытья поверхностей 5 л универсальное   
4274                                                                                Антисептик Prosept ECO ULTRA5л   
15888           Просепт Professional Multipower Prof средство усиленного действия для мытья полов концентрат (1 л)   
855                                                       Соль для посудомоечных машин PROSEPT Splash 1,5кг 280-15   
4607                               Концентрат для мытья полов и стен PROSEPTMultipower "После дождя" 0,8 л. 283-08   

       product_id  \
1733          257   
1133          275   
5736           61   
8804          191   
14586          56   
12367         305   
11487         266   
2564          279   
10719         322   
9555          386   
8053          150   
10537         324   
2806           72   
11169         240   
6378          254   
5094           13   
4274          246   
15888          24   
855           106   
4607           44   

                                                                                                                         original_name  \
1733                                                              Антисептик для внутренних работPROSEPT INTERIORконцентрат 1:10 / 1 л   
1133                                                                                             Антисептик ECO ULTRA коричневый / 5 л   
5736                                                           Средство для устранения засоров в трубахBath Profконцентрат 1:100 / 5 л   
8804                                                                Дезинфицирующее средство ПРОФ ДЗ "PROF DZ" готовый состав / 0,5 л    
14586  Гелеобразное средство усиленного действия для удаления ржавчины и минеральных отложенийBath Extraконцентрат 1:10-1:100 / 0,75 л   
12367                                                       Огнебиозащита   ОГНЕБИО PROF l, 1 группа, бесцветный готовый состав / 65 л   
11487                                                                 Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 5 л   
2564                                                                         

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 17566
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              17567 non-null  object
 1   product_id                17567 non-null  int64 
 2   original_name             17567 non-null  object
 3   product_name_normalized   17567 non-null  object
 4   original_name_normalized  17567 non-null  object
dtypes: int64(1), object(4)
memory usage: 823.5+ KB


In [31]:
# Подсчет количества дубликатов
num_duplicates = df.duplicated().sum()

print(f"Количество дубликатов в датафрейме: {num_duplicates}")

Количество дубликатов в датафрейме: 15877


In [32]:
# Удаление полных дубликатов
df = df.drop_duplicates()

## Создаем эмбеддинги наименований при помощи двуязычной версии BERT.

In [33]:
# Загрузка мультиязычного токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [34]:
# Функция для подсчета длин токенов
def get_token_lengths(column):
    token_lengths = [len(tokenizer.tokenize(text)) for text in df[column]]
    return np.mean(token_lengths), np.median(token_lengths), np.max(token_lengths)

# Подсчитываем длину токенизированных текстов для каждого столбца
avg_length_product, median_length_product, max_length_product = get_token_lengths('product_name_normalized')
avg_length_original, median_length_original, max_length_original = get_token_lengths('original_name_normalized')

# Вывод статистики
print(f"Для 'product_name_normalized': Средняя длина: {avg_length_product}, Медианная длина: {median_length_product}, Максимальная длина: {max_length_product}")
print(f"Для 'original_name_normalized': Средняя длина: {avg_length_original}, Медианная длина: {median_length_original}, Максимальная длина: {max_length_original}")

Для 'product_name_normalized': Средняя длина: 14.44733727810651, Медианная длина: 14.0, Максимальная длина: 36
Для 'original_name_normalized': Средняя длина: 15.03491124260355, Медианная длина: 14.0, Максимальная длина: 30


In [35]:
# Функция для генерации эмбеддингов
def generate_embeddings(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=36, padding='max_length',
                              truncation=True, add_special_tokens=True, return_attention_mask=True)
    # Выключение расчета градиентов для ускорения процесса
    with torch.no_grad():
        # Получение выходных данных модели
        output = model(**encoded_input)
    # Возвращение усредненных эмбеддингов последнего слоя
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

In [36]:
%%time

# Генерация эмбеддингов для каждого столбца с индикатором прогресса
df['product_name_embeddings'] = df['product_name_normalized'].progress_apply(generate_embeddings)
df['original_name_embeddings'] = df['original_name_normalized'].progress_apply(generate_embeddings)

100%|██████████| 1690/1690 [01:27<00:00, 19.37it/s]

CPU times: user 11min 44s, sys: 1.94 s, total: 11min 46s
Wall time: 2min 56s


In [37]:
# Проверка результатов
print(df[['product_name_embeddings', 'original_name_embeddings']].head())

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1690 entries, 0 to 17564
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              1690 non-null   object
 1   product_id                1690 non-null   int64 
 2   original_name             1690 non-null   object
 3   product_name_normalized   1690 non-null   object
 4   original_name_normalized  1690 non-null   object
 5   product_name_embeddings   1690 non-null   object
 6   original_name_embeddings  1690 non-null   object
dtypes: int64(1), object(6)
memory usage: 105.6+ KB


## Создание модели ранжирования и расчет ее точности.

Поскольку нам важно, чтобы в предложенных вариантах (желательно на первом месте) оказался верный вариант, то метрикой качества должна быть recall.

In [39]:
# Функция для расчета Recall at K
def recall_at_k(predicted, correct, k=5):
    return int(correct in predicted[:k])

In [40]:
%%time
# Собираем все уникальные эмбеддинги товаров с сайта заказчика
unique_original_embeddings = df.drop_duplicates(subset=['product_id'])[['product_id', 'original_name_embeddings']]
unique_embeddings_matrix = np.stack(unique_original_embeddings['original_name_embeddings'].values)

# Ранжирование товаров и подсчет Recall at 5
total_recall = 0
for index, row in df.iterrows():
    # Расчет расстояний
    distances = pdist(np.vstack([row['product_name_embeddings'], unique_embeddings_matrix]), 'cosine')
    distance_matrix = squareform(distances)[0, 1:]
    top_5_indices = np.argsort(distance_matrix)[:5]  # Топ-5 индексов с наименьшими расстояниями

    # Получаем ID топ-5 товаров
    top_5_products_ids = unique_original_embeddings.iloc[top_5_indices]['product_id'].values

    # Добавляем recall для текущего товара
    total_recall += recall_at_k(top_5_products_ids, row['product_id'])

average_recall = total_recall / len(df)
print(f"Средний Recall на Топ-5: {average_recall}")

Средний Recall на Топ-5: 0.8390532544378698
CPU times: user 1min 20s, sys: 44.1 ms, total: 1min 20s
Wall time: 1min 20s


## Демонстрация работы модели

In [41]:
import random

# Генерация случайного индекса
random_index = random.randint(0, len(df) - 1)
random_product = df.iloc[random_index]

print(f"Выбранный товар: {random_product['product_name']} (ID: {random_product['product_id']})")

# Расчет расстояний для выбранного товара
distances = pdist(np.vstack([random_product['product_name_embeddings'], unique_embeddings_matrix]), 'euclidean')
distance_matrix = squareform(distances)[0, 1:]
top_5_indices = np.argsort(distance_matrix)[:5]  # Топ-5 индексов с наименьшими расстояниями

# Получение рекомендаций
top_5_recommendations = unique_original_embeddings.iloc[top_5_indices]
print("\nТоп-5 рекомендованных товаров:")
for idx, row in top_5_recommendations.iterrows():
    print(f"ID: {row['product_id']}, Название: {df[df['product_id'] == row['product_id']]['original_name'].iloc[0]}")

Выбранный товар: Средство для удаления плесени PROSEPT Bath Fungi с дезинфицирующим эффектом, 500 мл, спрей (ID: 67)

Топ-5 рекомендованных товаров:
ID: 67, Название: Средство для удаления плесени  с дезинфицирующим эффектомBath Fungi концентрат 1:50-1:100 / 0,5 л
ID: 68, Название: Средство для удаления плесени  с дезинфицирующим эффектомBath Fungi концентрат 1:50-1:100 / 5 л
ID: 55, Название: Средство усиленного действия для удаления ржавчины и минеральных отложенийBath Acid +  с ароматом цитрусаконцентрат 1:200-1:500 / 0,75 л
ID: 107, Название: Средство для мытья и отбеливания посуды с дезинфицирующим эффектом.Cooky Whiteконцентрат 1:50-1:100 / 5 л
ID: 126, Название: Средство для комплексного мытья и отбеливания поверхностей с дезинфицирующим эффектом.Duty Beliznaконцентрат 1:200-1:3000 / 5 л 


In [42]:
%%time
# Инициализация списка для индексов строк с recall = 0
rows_with_zero_recall = []

# Определение строк, где recall равен 0
for index, row in df.iterrows():
    # Расчет расстояний
    distances = pdist(np.vstack([row['product_name_embeddings'], unique_embeddings_matrix]), 'cosine')
    distance_matrix = squareform(distances)[0, 1:]
    top_5_indices = np.argsort(distance_matrix)[:5]  # Топ-5 индексов с наименьшими расстояниями

    # Получаем ID топ-5 товаров
    top_5_products_ids = unique_original_embeddings.iloc[top_5_indices]['product_id'].values

    # Проверка наличия верного товара среди топ-5
    if row['product_id'] not in top_5_products_ids:
        rows_with_zero_recall.append(index)

# Вывод первых 10 строк с recall = 0, исключая столбцы с эмбеддингами
print("Товары, для которых система не предложила верного варианта:")
columns_to_display = [col for col in df.columns if 'embeddings' not in col]
display(df.loc[rows_with_zero_recall, columns_to_display].head(10))

Товары, для которых система не предложила верного варианта:


product_name  \
44                             Средство чистящее универсальное PROSEPT 500 мл   
237                     Средство для мытья пола Prosept Multipower Citrus 1 л   
841            Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.   
1385                    СредстводляуборкипослестроительстваProseptDutyExtra1л   
1550                            АнтисептикневымываемыйProseptEcoUltra,орех,5л   
1553                     Антисептик невымываемый Prosept Eco Ultra, орех, 5 л   
1610                            Строительный антисептик  PROSEPT Невымываемый   
1617  Огнебиозащита PROSEPT prof 1 группа для наружных и внутренних работ 10л   
1623                  ОгнебиозащитадлядревесиныProseptОГНЕБИОPROFI,10л,007-10   
1626         Огнебиозащита для древесины Prosept ОГНЕБИО PROF I, 10 л, 007-10   

      product_id  \
44            12   
237           38   
841          236   
1385         123   
1550         250   
1553         250   
1610         250   
1617         307   
1623         307   
1626         307   

                                                                      original_name  \
44    Универсальное моющее и чистящее средствоUniversal Sprayготовый состав / 0,5 л   
237         Концентрат для мытья половMULTIPOWER с ароматом цитрусаконцентрат / 1 л   
841          Шпатлевка-замазка для заделки глубоких выбоин и трещинPlastix / 1,4 кг   
1385   Средство для уборки после строительстваDuty Extraконцентрат 1:2-1:100 / 1 л    
1550           Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 5 л   
1553           Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 5 л   
1610           Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 5 л   
1617          Огнебиозащита ОГНЕБИО PROF l, 1 группа, красный готовый состав / 10 л   
1623          Огнебиозащита ОГНЕБИО PROF l, 1 группа, красный готовый состав / 10 л   
1626          Огнебиозащита ОГНЕБИО PROF l, 1 группа, красный готовый состав / 10 л   

                                            product_name_normalized  \
44                                                           500 мл   
237                                      пола multipower citrus 1 л   
841            шпаклевка выравнивающая акриловая plastix белая 1 кг   
1385                         уборкипослестроительства dutyextra 1 л   
1550                             антисептик ecoultra коричневый 5 л   
1553                             антисептик ecoultra коричневый 5 л   
1610                                        строительный антисептик   
1617  огнебиозащита prof 1-ая группа наружных внутренних работ 10 л   
1623                     огнебиозащита древесины огнебио profi 10 л   
1626                    огнебиозащита древесины огнебио prof i 10 л   

                                             original_name_normalized  
44                                             universal spray 500 мл  
237                           полов multipower с ароматом цитруса 1 л  
841   шпатлевка-замазка заделки глубоких выбоин трещин plastix 1400 г  
1385                        уборки после строительства duty extra 1 л  
1550                                  антисептик ultra коричневый 5 л  
1553                                  антисептик ultra коричневый 5 л  
1610                                  антисептик ultra коричневый 5 л  
1617            огнебиозащита огнебио prof l 1-ая группа красный 10 л  
1623            огнебиозащита огнебио prof l 1-ая группа красный 10 л  
1626            огнебиозащита огнебио prof l 1-ая группа красный 10 л

CPU times: user 1min 21s, sys: 59 ms, total: 1min 21s
Wall time: 1min 20s
